In [1]:
%pwd

'd:\\LLMs\\CrewAI-Finance-Agent\\experiments'

In [2]:
import os
os.chdir("../")
%pwd

'd:\\LLMs\\CrewAI-Finance-Agent'

In [3]:
from crewai import Agent
from dotenv import load_dotenv
load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI
import os
import google.generativeai as genai
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))



llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash",
                           verbose=True,
                           temperature=0.5,
                           convert_system_message_to_human=True,google_api_key=os.getenv("GOOGLE_API_KEY"))


support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at Google Cloud (https://cloud.google.com/) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	verbose=True,
    llm=llm
    ,allow_delegation=True,
)


support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://cloud.google.com/) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True,
    llm = llm
    ,allow_delegation = False,
)



d:\LLMs\CrewAI-Finance-Agent\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from crewai_tools import SerperDevTool,ScrapeWebsiteTool, WebsiteSearchTool, DirectoryReadTool, FileReadTool


docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://cloud.google.com/speech-to-text/docs/speech-to-text-requests"
)

In [5]:
from crewai import Task
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)



quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)

In [6]:
from crewai import Crew
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
  memory=True
)



inputs = {
    "customer": "CampusX",
    "person": "Md Amanatullah",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}



result = crew.kickoff(inputs=inputs)
print(result)

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: CampusX just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Md Amanatullah from CampusX is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: fake. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [8]:

## https://serper.dev/

from dotenv import load_dotenv
load_dotenv()
import os

os.environ['SERPER_API_KEY'] = os.getenv('SERPER_API_KEY')


from crewai_tools import SerperDevTool

# Initialize the tool for internet searching capabilities
tool = SerperDevTool()

In [9]:
from crewai import Agent
from dotenv import load_dotenv
load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI
import os


## call the gemini models
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash",
                           verbose=True,
                           temperature=0.5,
                           google_api_key=os.getenv("GOOGLE_API_KEY"))

# Creating a senior researcher agent with memory and verbose mode

news_researcher=Agent(
    role="Senior Researcher",
    goal='Unccover ground breaking technologies in {topic}',
    verbose=True,
    memory=True,
    backstory=(
        "Driven by curiosity, you're at the forefront of"
        "innovation, eager to explore and share knowledge that could change"
        "the world."

    ),
    tools=[tool],
    llm=llm,
    allow_delegation=True

)

## creating a write agent with custom tools responsible in writing news blog

news_writer = Agent(
  role='Writer',
  goal='Narrate compelling tech stories about {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "With a flair for simplifying complex topics, you craft"
    "engaging narratives that captivate and educate, bringing new"
    "discoveries to light in an accessible manner."
  ),
  tools=[tool],
  llm=llm,
  allow_delegation=False
)

In [11]:

from crewai import Task
# Research task
research_task = Task(
  description=(
    "Identify the next big trend in {topic}."
    "Focus on identifying pros and cons and the overall narrative."
    "Your final report should clearly articulate the key points,"
    "its market opportunities, and potential risks."
  ),
  expected_output='A comprehensive 3 paragraphs long report on the latest AI trends.',
  tools=[tool],
  agent=news_researcher,
)

# Writing task with language model configuration
write_task = Task(
  description=(
    "Compose an insightful article on {topic}."
    "Focus on the latest trends and how it's impacting the industry."
    "This article should be easy to understand, engaging, and positive."
  ),
  expected_output='A 4 paragraph article on {topic} advancements formatted as markdown.',
  tools=[tool],
  agent=news_writer,
  async_execution=False,
  output_file='new-blog-post.md'  # Example of output customization
)


In [12]:
from crewai import Crew,Process

## Forming the tech focused crew with some enhanced configuration
crew=Crew(
    agents=[news_researcher,news_writer],
    tasks=[research_task,write_task],
    process=Process.sequential,

)

## starting the task execution process wiht enhanced feedback

result=crew.kickoff(inputs={'topic':'AI in healthcare'})
print(result)

2024-06-02 18:54:28,950 - 28564 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed




> Entering new CrewAgentExecutor chain...
Thought: I need to understand the current landscape of AI in healthcare to identify the next big trend. 

Action: Search the internet
Action Input: { "search_query": "AI trends in healthcare" } 
 


Search results: Title: Five trends converging for AI-enabled healthcare: Why AI and ... - PwC
Link: https://www.pwc.com/gx/en/industries/healthcare/publications/ai-robotics-new-health/five-trends.html
Snippet: Five distinct trends are converging, which mean AI and robotics will come to define New Health: First and foremost is the value challenge that all countries ...
---
Title: AI in 2024: Welcome to the 'new normal' in healthcare
Link: https://www.healthcareitnews.com/news/ai-2024-welcome-new-normal-healthcare
Snippet: Regulators are starting to see an opportunity to incentivize the use of artificial intelligence and machine learning, says one expert, who ...
---
Title: Artificial intelligence in medicine: current trends and future possibilities

2024-06-02 18:55:22,800 - 28564 - before_sleep.py-before_sleep:65 - WARNING: Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
2024-06-02 18:55:25,465 - 28564 - before_sleep.py-before_sleep:65 - WARNING: Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


Thought: I need to gather information on the pros, cons, market opportunities, and potential risks of Generative AI in personalized medicine. 

Action: Search the internet
Action Input: { "search_query": "Generative AI in personalized medicine pros and cons" } 


Search results: Title: Pros & Cons of Artificial Intelligence in Medicine | Drexel CCI
Link: https://drexel.edu/cci/stories/artificial-intelligence-in-medicine-pros-and-cons/
Snippet: Innovations include appointment-scheduling, translating clinical details and tracking patient histories. AI is enabling healthcare facilities to ...
---
Title: Top 3 Pros and Cons of Generative AI in Medical Record Keeping
Link: https://augmedix.com/resources/blog/top-3-pros-and-cons-of-generative-ai-in-medical-record-keeping/
Snippet: Top 3 Pros of Generative AI in Medical Record Keeping · 1. Reducing Time on the EHR and Other Administrative Tasks · 2. Alleviating Clinician ...
---
Title: Exploring generative artificial intelligence in healthcar